<a href="https://colab.research.google.com/github/YaminMahdi/All-.ipynb/blob/master/LAB4_193_15_1071.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q gdown && gdown --fuzzy https://drive.google.com/file/d/1HFIClmLVILw851oU3n-hAz3zXkcMZK-I/view      
!tar xf spark-3.1.2-bin-hadoop3.2_by_yk.tgz    
!pip install -q findspark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Downloading...
From: https://drive.google.com/uc?id

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("RDD").getOrCreate()
spark

In [29]:
!wget -q https://raw.githubusercontent.com/YaminMahdi/All-.ipynb/master/Linear_regression_dataset.csv
df = spark.read.csv("/content/Linear_regression_dataset.csv", header = True, inferSchema = True)
df.show(5)

+-----+-----+-----+-----+-----+------+
|var_1|var_2|var_3|var_4|var_5|output|
+-----+-----+-----+-----+-----+------+
|  734|  688|   81|0.328|0.259| 0.418|
|  700|  600|   94| 0.32|0.247| 0.389|
|  712|  705|   93|0.311|0.247| 0.417|
|  734|  806|   69|0.315| 0.26| 0.415|
|  613|  759|   61|0.302| 0.24| 0.378|
+-----+-----+-----+-----+-----+------+
only showing top 5 rows



In [30]:
from pyspark.sql.functions import corr
df.select(corr('var_1','output')).show()

+-------------------+
|corr(var_1, output)|
+-------------------+
| 0.9187399607627283|
+-------------------+



In [31]:
df.select(corr('var_1', 'var_2')).show()
df.select(corr('var_2', 'output')).show()


+------------------+
|corr(var_1, var_2)|
+------------------+
|0.3801386662491575|
+------------------+

+-------------------+
|corr(var_2, output)|
+-------------------+
|0.43652698913681093|
+-------------------+



In [15]:
df.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)



In [8]:
from pyspark.ml.feature import VectorAssembler
vec_assembler = VectorAssembler(inputCols= ['var_1', 'var_2', 'var_3', 'var_4', 'var_5'], outputCol="features")
features_df = vec_assembler.transform(df)
features_df.show(5, truncate = False)

+-----+-----+-----+-----+-----+------+------------------------------+
|var_1|var_2|var_3|var_4|var_5|output|features                      |
+-----+-----+-----+-----+-----+------+------------------------------+
|734  |688  |81   |0.328|0.259|0.418 |[734.0,688.0,81.0,0.328,0.259]|
|700  |600  |94   |0.32 |0.247|0.389 |[700.0,600.0,94.0,0.32,0.247] |
|712  |705  |93   |0.311|0.247|0.417 |[712.0,705.0,93.0,0.311,0.247]|
|734  |806  |69   |0.315|0.26 |0.415 |[734.0,806.0,69.0,0.315,0.26] |
|613  |759  |61   |0.302|0.24 |0.378 |[613.0,759.0,61.0,0.302,0.24] |
+-----+-----+-----+-----+-----+------+------------------------------+
only showing top 5 rows



In [9]:
model_df = features_df.select('features', 'output')
model_df.show(5, truncate= False)

+------------------------------+------+
|features                      |output|
+------------------------------+------+
|[734.0,688.0,81.0,0.328,0.259]|0.418 |
|[700.0,600.0,94.0,0.32,0.247] |0.389 |
|[712.0,705.0,93.0,0.311,0.247]|0.417 |
|[734.0,806.0,69.0,0.315,0.26] |0.415 |
|[613.0,759.0,61.0,0.302,0.24] |0.378 |
+------------------------------+------+
only showing top 5 rows



In [10]:
train_df, test_df = model_df.randomSplit([.8, .2])
print(train_df.count(), len(train_df.columns))

989 2


In [11]:
train_df.describe().show()
test_df.describe().show()

+-------+-------------------+
|summary|             output|
+-------+-------------------+
|  count|                989|
|   mean|0.39703640040444826|
| stddev|0.03338429000920117|
|    min|              0.301|
|    max|              0.491|
+-------+-------------------+

+-------+-------------------+
|summary|             output|
+-------+-------------------+
|  count|                243|
|   mean| 0.3985843621399177|
| stddev|0.03282353397405582|
|    min|              0.315|
|    max|              0.485|
+-------+-------------------+



In [16]:
from pyspark.ml.regression import LinearRegression
lin_reg= LinearRegression(labelCol='output')

In [20]:
lin1 = lin_reg.fit(train_df)
print(lin1.intercept)
print(lin1.coefficients)

0.1769695876577605
[0.000332570940544152,5.490620608029893e-05,0.00018615686578267497,-0.6204492923934785,0.5035517467324592]


In [24]:
train_pred = lin1.evaluate(train_df)
train_pred.meanSquaredError

0.00014806301891363794

In [27]:
test_pred = lin1.evaluate(test_df)
test_pred.meanSquaredError
#test_pred.rootMeanSquaredError

0.00013101015951456007